<a href="https://colab.research.google.com/github/puzzy8338/chatgpt-proxy/blob/main/00981A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd

# ✅ 優先搜尋的目錄（Google Drive）
PREFERRED_DIR = "/content/drive/MyDrive/股票/981a"

# ✅ 嘗試在 Colab 自動掛載 Google Drive（若路徑不可用）
def ensure_gdrive_mounted(preferred_dir=PREFERRED_DIR):
    try:
        # 僅在路徑不存在時嘗試掛載
        if not os.path.exists(preferred_dir):
            # 僅在 colab 執行時才有 google.colab
            from google.colab import drive  # type: ignore
            print("🔌 嘗試掛載 Google Drive ...")
            drive.mount("/content/drive")
    except Exception as e:
        # 非 Colab 或掛載失敗就略過，之後會走其他搜尋路徑
        print(f"⚠️ 無法自動掛載 Google Drive（可能非 Colab 環境）：{e}")

# ✅ 依照優先順序解析檔案完整路徑
def resolve_file(date_str):
    fname = f"ETF_Investment_Portfolio_{date_str}.xlsx"
    # 1) 優先找 Google Drive 目錄
    preferred_path = os.path.join(PREFERRED_DIR, fname)
    if os.path.exists(preferred_path):
        return preferred_path

    # 2) 次之找目前工作目錄
    cwd_path = os.path.join(os.getcwd(), fname)
    if os.path.exists(cwd_path):
        return cwd_path

    # 3) 都找不到就要求上傳
    try:
        from google.colab import files  # type: ignore
        print(f"🔍 找不到檔案：{preferred_path} 或 {cwd_path}")
        print(f"⬆️ 請上傳檔案：{fname}")
        uploaded = files.upload()
        if fname not in uploaded:
            raise FileNotFoundError(f"❌ 未上傳必要檔案：{fname}")
        # 上傳後檔案會存到目前工作目錄
        return os.path.join(os.getcwd(), fname)
    except Exception as e:
        # 非 Colab 或者使用者沒有上傳
        raise FileNotFoundError(
            f"❌ 無法取得檔案：{fname}\n"
            f"請將檔案放在 {PREFERRED_DIR} 或目前目錄，或在 Colab 直接上傳。\n原始錯誤：{e}"
        )

# ✅ 自動偵測表頭列
def find_stock_header_index(df_raw):
    # 需要同時包含以下關鍵欄位（可分散在同一列不同儲存格）
    keywords = ['股票', '股票代號', '股票名稱', '股數']
    for idx, row in df_raw.iterrows():
        cells = row.astype(str).tolist()
        if all(any(k in c for c in cells) for k in keywords):
            return idx
    return None

def main():
    # 先嘗試掛載 Google Drive（在 Colab 時）
    ensure_gdrive_mounted()

    # ✅ 使用者輸入兩個日期（格式：YYYYMMDD）
    date_today = input("請輸入『今日』的日期（格式：YYYYMMDD）：").strip()
    date_old = input("請輸入『比較日』的日期（格式：YYYYMMDD）：").strip()

    # ✅ 解析檔案完整路徑（優先 Google Drive）
    file_today = resolve_file(date_today)
    file_old = resolve_file(date_old)

    print(f"📂 今日檔案：{file_today}")
    print(f"📂 比較日檔案：{file_old}")

    # ✅ 載入原始檔案（不設欄位）
    df_today_raw = pd.read_excel(file_today, header=None)
    df_old_raw = pd.read_excel(file_old, header=None)

    header_idx_today = find_stock_header_index(df_today_raw)
    header_idx_old = find_stock_header_index(df_old_raw)

    if header_idx_today is None or header_idx_old is None:
        raise Exception("❌ 找不到包含「股票代號」「股票名稱」「股數」等欄位的表格，請檢查 Excel 格式")

    # ✅ 重新讀取並處理必要欄位
    df_today = pd.read_excel(file_today, header=header_idx_today)
    df_old = pd.read_excel(file_old, header=header_idx_old)
    required_cols = ['股票代號', '股票名稱', '股數']

    if not all(col in df_today.columns for col in required_cols) or not all(col in df_old.columns for col in required_cols):
        raise Exception("❌ 缺少必要欄位：股票代號、股票名稱、股數")

    # ✅ 清理空格、欄位重新命名
    for df in [df_today, df_old]:
        df['股票代號'] = df['股票代號'].astype(str).str.strip()
        df['股票名稱'] = df['股票名稱'].astype(str).str.strip()

    df_today = df_today[required_cols].copy()
    df_today.rename(columns={'股數': '股數_new', '股票名稱': '股票名稱_new'}, inplace=True)

    df_old = df_old[required_cols].copy()
    df_old.rename(columns={'股數': '股數_old', '股票名稱': '股票名稱_old'}, inplace=True)

    # ✅ 合併、轉換股數、計算變化
    merged = pd.merge(df_old, df_today, on='股票代號', how='outer')
    merged['股票名稱'] = merged['股票名稱_new'].combine_first(merged['股票名稱_old'])

    merged['股數_old'] = pd.to_numeric(
        merged['股數_old'].astype(str).str.replace(',', ''),
        errors='coerce'
    ).fillna(0)
    merged['股數_new'] = pd.to_numeric(
        merged['股數_new'].astype(str).str.replace(',', ''),
        errors='coerce'
    ).fillna(0)
    merged['股數變化'] = merged['股數_new'] - merged['股數_old']

    # ✅ 輸出結果
    result = merged[['股票代號', '股票名稱', '股數_old', '股數_new', '股數變化']].copy()
    result.columns = ['股票代號', '股票名稱', '前股數', '今股數', '股數變化']

    output_file = f"stock_changes_{date_old}_to_{date_today}.xlsx"
    result.to_excel(output_file, index=False)

    print("✅ 報告已產生：", output_file)

    # 若在 Colab，提供下載
    try:
        from google.colab import files  # type: ignore
        print("⬇️ 請點擊下方下載：")
        files.download(output_file)
    except Exception:
        # 非 Colab 就不做下載
        pass

if __name__ == "__main__":
    main()


請輸入『今日』的日期（格式：YYYYMMDD）：20251021
請輸入『比較日』的日期（格式：YYYYMMDD）：20251020
📂 今日檔案：/content/drive/MyDrive/股票/981a/ETF_Investment_Portfolio_20251021.xlsx
📂 比較日檔案：/content/drive/MyDrive/股票/981a/ETF_Investment_Portfolio_20251020.xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ 報告已產生： stock_changes_20251020_to_20251021.xlsx
⬇️ 請點擊下方下載：


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')